# What is a Trigger?
- a function automatically used when a specific event occurs. Event can be INSERT, UPDATE, DELETE, or TRUNCATE. We can put trigger BEFORE or AFTER the event
- Useful if you want to keep track of the specific history of data
- two types
    - row trigger: trigger invoked each row change
    - statement trigger: trigger invoked each statement
- When can User Defined Functions AKA Stored Procedures be used?
    - when you are frequently calling thousands of rows 


1. create table: cpu_sales_update
3. create function: log_sales_data
2. create trigger: cpu_sales_update_trigger

In [1]:
import psycopg2           
import psycopg2.extras
import pandas as pd   
import numpy as np
import io
from faker import Factory,Faker # Create fake data
# Import the 'config' function from the config.py file:
from config import config


# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

In [3]:
# CREATE TABLE FOR PSQL: cpu_sales_update (used to log updated sales)

def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS cpu_sales_update;
        CREATE UNLOGGED TABLE cpu_sales_update (
            changed_on TIMESTAMP(6) NOT NULL,
            credit_card         TEXT NOT NULL,
            email               TEXT NOT NULL,
            first_name          TEXT NOT NULL,
            last_name           TEXT NOT NULL,
            primary_phone       TEXT NOT NULL,
            cpu                 TEXT NOT NULL,
            purchase_date       DATE NOT NULL
        );""")
    

with conn.cursor() as cursor:
    create_staging_table(cursor)

In [4]:
# Create Function to Log Updated Sales
    # Create function: log_sales_data
    # What to do when an event occurs
def create_log_func(cursor):
    cursor.execute("""
    CREATE OR REPLACE FUNCTION log_sales_data()
    RETURNS TRIGGER AS $cpu_sales_update$
        BEGIN
            --
            -- Create a new row in cpu_sales_update, for operation performed on 
            -- 'ppl_cpu_purchases'. (TG_OP) is used to designate what operation is
            -- performed and triggers your 'TRIGGER'

            IF (TG_OP = 'DELETE') THEN 
                INSERT INTO cpu_sales_update SELECT now(), OLD.*;
                RETURN OLD;

            ELSEIF (TG_OP = 'UPDATE') THEN
                INSERT INTO cpu_sales_update SELECT now(),NEW.*;
                RETURN NEW;

            ELSIF (TG_OP = 'INSERT') THEN
                INSERT INTO cpu_sales_update SELECT now(),NEW.*;

            END IF;
            RETURN NULL; -- result is ignored since this is an AFTER trigger
        
        END;    
    $cpu_sales_update$ LANGUAGE plpgsql;""")

# Send Over for Function which will store trigger information
with conn.cursor() as cursor:
    create_log_func(cursor)

In [5]:
# Trigger Creation: which will start (AFTER) the event!
    # Trigger name: cpu_sales_update_trigger
    # trigger When event happened in 'ppl_cpu_purchases' table

def trigger_cpu_sales(cursor):
    cursor.execute("""
    CREATE TRIGGER cpu_sales_update_trigger
    AFTER INSERT OR UPDATE OR DELETE ON ppl_cpu_purchases
    FOR EACH ROW EXECUTE PROCEDURE log_sales_data();
    """)

# Send Trigger to PSQL:
with conn.cursor() as cursor:
    trigger_cpu_sales(cursor)   

DuplicateObject: trigger "cpu_sales_update_trigger" for relation "ppl_cpu_purchases" already exists


In [6]:
# All Triggers in Current DB:
    # location: postgres/Catalogs/PostgreSQL Catalog (pg_catalog)/Tables/pg_trigger
cur.execute('SELECT * FROM pg_trigger;')
cur.fetchall()

[(16996,
  16977,
  0,
  'cpu_sales_update_trigger',
  16919,
  29,
  'O',
  False,
  0,
  0,
  0,
  False,
  False,
  0,
  '',
  None,
  None,
  None)]

In [7]:
# Find All the Triggers I made on a Specific Table:
cur.execute("SELECT tgname FROM pg_trigger, \
pg_class WHERE tgrelid=pg_class.oid AND relname='ppl_cpu_purchases';")

cur.fetchall()

[('cpu_sales_update_trigger',)]

In [8]:
# Insert
cur.execute("INSERT INTO ppl_cpu_purchases (credit_card,email,first_name,last_name, \
primary_phone,cpu,purchase_date) VALUES ('9999-9999-9999-9999','MrFugu@gmail.com', \
'MrFugu','DataScience','1111111111', 'Intel Core i9-9999K','2020-05-05')")

# Record of Insert
cur.execute('SELECT * FROM cpu_sales_update')
cur.fetchall()

[(datetime.datetime(2020, 10, 21, 17, 2, 43, 978840),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFugu',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5))]

In [9]:
# Update 
cur.execute("UPDATE ppl_cpu_purchases SET first_name= 'MrFuguYay' \
WHERE first_name='MrFugu'")

# Record of Insert and Update; cpu_sales_update
cur.execute('SELECT * FROM cpu_sales_update')
cur.fetchall()

[(datetime.datetime(2020, 10, 21, 16, 44, 50, 982877),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFugu',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5)),
 (datetime.datetime(2020, 10, 21, 16, 45, 52, 786513),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFuguYay',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5)),
 (datetime.datetime(2020, 10, 21, 17, 2, 14, 605130),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFugu',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5)),
 (datetime.datetime(2020, 10, 21, 17, 2, 16, 661929),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFuguYay',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5))]

In [8]:
# Drop Trigger before you drop table

# cur.execute("DROP TRIGGER IF EXISTS cpu_sales_update ON ppl_cpu_purchases;")
# cur.execute("DROP TABLE IF EXISTS cpu_sales_update;")

In [ ]:
# Disable Trigger

# cur.execute("ALTER TABLE cpu_sales_update DISABLE TRIGGER cpu_sales_update_trigger")
# cur.execute("ALTER TABLE cpu_sales_update DISABLE TRIGGER ALL")